In [1]:
import numpy as np

## First part

In [2]:
# Read the puzzle, it contains a list of initial ages of fishes
t0_raw = []
#with open('fish_example.txt') as f:
with open('fish.txt') as f:
    t0_raw = f.readlines()
print(t0_raw)

# Convert from string to a list of initial ages
t0s = t0_raw[0].strip('\n').split(',')
t0s = [int(t0) for t0 in t0s]
print(t0s)

['2,5,5,3,2,2,5,1,4,5,2,1,5,5,1,2,3,3,4,1,4,1,4,4,2,1,5,5,3,5,4,3,4,1,5,4,1,5,5,5,4,3,1,2,1,5,1,4,4,1,4,1,3,1,1,1,3,1,1,2,1,3,1,1,1,2,3,5,5,3,2,3,3,2,2,1,3,1,3,1,5,5,1,2,3,2,1,1,2,1,2,1,2,2,1,3,5,4,3,3,2,2,3,1,4,2,2,1,3,4,5,4,2,5,4,1,2,1,3,5,3,3,5,4,1,1,5,2,4,4,1,2,2,5,5,3,1,2,4,3,3,1,4,2,5,1,5,1,2,1,1,1,1,3,5,5,1,5,5,1,2,2,1,2,1,2,1,2,1,4,5,1,2,4,3,3,3,1,5,3,2,2,1,4,2,4,2,3,2,5,1,5,1,1,1,3,1,1,3,5,4,2,5,3,2,2,1,4,5,1,3,2,5,1,2,1,4,1,5,5,1,2,2,1,2,4,5,3,3,1,4,4,3,1,4,2,4,4,3,4,1,4,5,3,1,4,2,2,3,4,4,4,1,4,3,1,3,4,5,1,5,4,4,4,5,5,5,2,1,3,4,3,2,5,3,1,3,2,2,3,1,4,5,3,5,5,3,2,3,1,2,5,2,1,3,1,1,1,5,1\n']
[2, 5, 5, 3, 2, 2, 5, 1, 4, 5, 2, 1, 5, 5, 1, 2, 3, 3, 4, 1, 4, 1, 4, 4, 2, 1, 5, 5, 3, 5, 4, 3, 4, 1, 5, 4, 1, 5, 5, 5, 4, 3, 1, 2, 1, 5, 1, 4, 4, 1, 4, 1, 3, 1, 1, 1, 3, 1, 1, 2, 1, 3, 1, 1, 1, 2, 3, 5, 5, 3, 2, 3, 3, 2, 2, 1, 3, 1, 3, 1, 5, 5, 1, 2, 3, 2, 1, 1, 2, 1, 2, 1, 2, 2, 1, 3, 5, 4, 3, 3, 2, 2, 3, 1, 4, 2, 2, 1, 3, 4, 5, 4, 2, 5, 4, 1, 2, 1, 3, 5, 3, 3, 5, 4, 1, 1, 5, 2, 4, 4, 1, 

In [3]:
ts = t0s.copy()
number_of_days = 100
for n in range(number_of_days):
    ts_new = ts
    #print('After ' + str(n) + ' days: ' + str(ts_new) )
    for i,t in enumerate(ts):
        if t == 0:
            ts_new.append(9)
            ts_new[i] = 6
        elif t!= 0:
            ts_new[i] = t-1
            
    ts = ts_new.copy()

In [4]:
print(f'Number of fishes after {number_of_days} days: {len(ts_new)}')

Number of fishes after 100 days: 2012396


## Second part

The previous formalism is too slow for calculating the number of fishes after 256 days! 
A smarter approach: calculate only how many new fishes arise from an initial fish of a certain age after n_days. Then multiply by the total number of fishes with that initial age. 
Even smarter and faster: only count how many fishes have each of the 8 possible ages.

In [5]:
def new_fishes_fast(t0, n_days):
    # Fishes can have counters 0, 1, 2, ..., 7, 8 (in total, 8+1)
    max_counter = 8
    counters = np.zeros((max_counter+1), dtype=np.uint32)
    counters[t0] = 1
    counters_new = np.zeros((max_counter+1), dtype=np.uint32)
    for n in range(n_days):
        # All the fishes lose one day (move one place in the counter array),
        # except fishes with zero days which go to the 6th place (index 5).
        # Also new fishes in the last place are added equal to the number of 
        # fishes with counter = 0
        for i in range(max_counter+1):
            if i == 6:
                counters_new[i] = counters[i+1] + counters[0]
            elif i == max_counter:
                counters_new[i] = counters[0]
            else:
                counters_new[i] = counters[i+1]
        
        #Save the new counter for the next loop
        counters = np.copy(counters_new)
        #The number of fishes is the add of all fishes in every counter place
    return(sum(counters))

def new_fishes(t0, n_days):
    #ts = np.array([t0], dtype=np.uint8)
    ts = [t0]
    for n in range(n_days):
        ts_new = ts
        #print('After ' + str(n) + ' days: ' + str(ts_new) )
        for i,t in enumerate(ts):
            if t == 0:
                ts_new.append(9)
                ts_new[i] = 6
            elif t!= 0:
                ts_new[i] = t-1

        ts = ts_new.copy()
    return(len(ts))

In [6]:
total = 0
total_fast = 0
number_of_days = 256
for t0 in set(t0s):
    #total += new_fishes(t0, number_of_days) * t0s.count(t0)
    total_fast += new_fishes_fast(t0, number_of_days) * t0s.count(t0)
    
#print(f'Number of fishes after {number_of_days} days: {total}')
print(f'Number of fishes after {number_of_days} days: {total_fast}')

Number of fishes after 256 days: 1601616884019


In [7]:
number_of_days = 18
t0 = 2
for i in range(number_of_days):
    print(i, new_fishes(t0, i), new_fishes_fast(t0,i))
    
#f'Number of fishes after {number_of_days} days: {total}'

0 1 1
1 1 1
2 1 1
3 2 2
4 2 2
5 2 2
6 2 2
7 2 2
8 2 2
9 2 2
10 3 3
11 3 3
12 4 4
13 4 4
14 4 4
15 4 4
16 4 4
17 5 5


In [8]:
#class Fish:
#    """A Fish class"""
#    def __init__(self, age, timer):
#        self.age = age
#        self.timer = age
#        
#    def reproduce(self):
#        self.age += 1
#        if self.timer != 0:
#            self.timer -= 1
#        elif self.timer == 0:
#            self.timer = 6
#
#fishes = [Fish(t0, t0) for t0 in t0s]
#number_of_days = 18
#for i in range(number_of_days):
#    for fish in fishes:
#        fish.reproduce()